# Model 서브클래싱

In [11]:
from tensorflow import keras
from tensorflow.keras import layers


In [12]:
import numpy as np
vocabulary_size = 10000
num_tags = 100
num_departments = 4

num_samples = 1280

title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
text_body_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags))

priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0, 2, size=(num_samples, num_departments))

In [13]:
class CustomerTicketModel(keras.Model) :
    def __init__(self, num_departments) :
        #생성자 층정의
        super().__init__()
        self.concat_layer = layers.Concatenate()
        self.mixing_layer = layers.Dense(64, activation="relu")
        self.priority_scorer = layers.Dense(1, activation="sigmoid")
        self.department_classifier = layers.Dense(
            num_departments, activation="softmax")

    def call(self, inputs): #정방향 패스 정의
        title = inputs["title"]
        text_body = inputs["text_body"]
        tags = inputs["tags"]
        
    
        features = self.concat_layer([title, text_body, tags])
        features = self.mixing_layer(features)
        priority = self.priority_scorer(features)
        department = self.department_classifier(features)
        return priority, department

In [14]:
model = CustomerTicketModel(num_departments=4) 
priority, department = model(
    {"title": title_data, "text_body": text_body_data, "tags": tags_data})

In [15]:

model.compile(optimizer="rmsprop",
              loss=["mean_squared_error", "categorical_crossentropy"],
              metrics=[["mean_absolute_error"], ["accuracy"]])
model.fit({"title": title_data,
           "text_body": text_body_data,
           "tags": tags_data},
          [priority_data, department_data],
          epochs=1)
model.evaluate({"title": title_data,
                "text_body": text_body_data,
                "tags": tags_data},
               [priority_data, department_data])
priority_preds, department_preds = model.predict({"title": title_data,
                                                  "text_body": text_body_data,
                                                  "tags": tags_data})

40/40 [==============================] - 1s 11ms/step - loss: 48.3659 - output_1_loss: 0.3173 - output_2_loss: 48.0486 - output_1_mean_absolute_error: 0.4837 - output_2_accuracy: 0.2609
